## Coding Quiz

With the given dataset, Please compare your best possible version of

    (1) BiLSTM,
    (2) BiLSTM with multiplicative attention (you have to fix e), and
    (3) BERT

Report the accuracy, precision, recall, and f1-score of each model.

For (1) and (2), use the following hyperparameters:

    Optimizer: SG
    Embedding: GloVe (https://pytorch.org/text/stable/vocab.html#torchtext.vocab.GloVe) >> Please change the embed_dim accordingly.
    Epochs: 2
    Batch size: 32
    Save the model with the best params

Anything not stated, please assume accordingly


For (2), Multiplicative attention differs from the General Attention (in Assignment 4) such that, for the *Alignment Scores* (or Energy), we multiply the Keys with some weights first before we dot the Keys with the Query.

$\mathbf{e}_i = \mathbf{q}^T \ \mathbf{W}  \mathbf{k}_t $

where $ \mathbf{W} \in \mathbb{R}^{h,h}$

* Hint : The shape of the Keys before and after multiplying with the weights should be the same

For (3), use this tutorial https://huggingface.co/docs/transformers/training as your guide.

In [ ]:
import os

os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [ ]:
import torchtext
import torch
from torch import nn
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### 1. Load the IMDB Review dataset from TorchText (https://pytorch.org/text/stable/datasets.html#id10)

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 1 if x == 'pos' else 0

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence #++

def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        length_list.append(processed_text.size(0))  #++<-----packed padded sequences require length
    #criterion expects float labels
    return torch.tensor(label_list, dtype=torch.float64), pad_sequence(text_list, padding_value=pad_idx, batch_first=True), torch.tensor(length_list, dtype=torch.int64)

from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.15)
num_val = int(len(train_dataset) * 0.10)
num_test = int(len(test_dataset) * 0.05)

split_train_, split_valid_, _ = \
    random_split(train_dataset, [num_train, num_val,len(train_dataset)- num_train - num_val])

split_test_, _ = \
    random_split(train_dataset, [num_test, len(test_dataset) - num_test])

train_loader = DataLoader(split_train_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(split_valid_, batch_size=batch_size,
                              shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(split_test_, batch_size=batch_size,
                             shuffle=True, collate_fn=collate_batch)